In [13]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql import types as T
from pyspark.sql import DataFrame as SparkDataFrame

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/11 09:19:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
train_raw : SparkDataFrame = spark.read.csv("../raw/train.csv", header=True)
test_raw : SparkDataFrame = spark.read.csv("../raw/test.csv", header = True)
store_raw : SparkDataFrame  = spark.read.csv("../raw/store.csv", header=True)

In [10]:
train_raw.describe().toPandas()

23/05/11 09:24:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,count,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209
1,mean,558.4297268309659,3.998340557348588,None,5773.818972305593,633.1459464082602,0.8301066939045958,0.3815145166824124,0.0,0.17864666946517382
2,stddev,321.9086511434528,1.9973909649405297,None,3849.9261752347525,464.4117338866323,0.3755392246931946,0.48575860487741956,0.0,0.38305636818092675
3,min,1,1,2013-01-01,0,0,0,0,0,0
4,max,999,7,2015-07-31,9999,999,1,1,c,1


In [12]:
train_raw.select(F.col("Store")).distinct().toPandas()

,Store
0,296
1,467
2,675
3,691
4,829
...,...
1110,119
1111,898
1112,1001
1113,438
